In [30]:
import pandas as pd
import numpy as np

raw_data = pd.read_csv("bank.csv", sep=';')
raw_data.info()
raw_data.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   object
dtypes: int64(7), object(10)
memory usage: 600.6+ KB


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no


In [31]:
raw_data = raw_data.drop(columns='duration')
raw_data.head(2)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,1,339,4,failure,no


In [32]:
from sklearn.preprocessing import LabelEncoder # Chuyển thuộc tính dạng category thành số.
def categoryToInt(data: pd.DataFrame):
    listAttrCate = ['job', 'marital', 'education', 'default', 'housing', 'month', 'loan', 'contact', 'poutcome', 'y']
    new_data = data.copy()
    LE = LabelEncoder()
    for attr in listAttrCate:
        new_data[attr] = LE.fit_transform(new_data[attr])
    return new_data

new_data = categoryToInt(raw_data)

In [33]:
from scipy import stats

z = np.abs(stats.zscore(new_data))
revmove_outlier_data = new_data[(z < 3).all(axis=1)]
revmove_outlier_data.shape

(3986, 16)

In [34]:
X = revmove_outlier_data.values[:,:-1]
y = revmove_outlier_data.values[:,-1:].flatten()# flatten copy còn ravel không copy
listAttrName = revmove_outlier_data.columns.tolist() # lấy danh sách tên thuộc tính (vẽ trong decision tree)
print('X[0]: ', *X[0])
print('len X[0]: ', len(X[0]))
print('Class 0: ', len(y[y==0]))
print('Class 1: ', len(y[y==1]))

X[0]:  30 10 1 0 0 1787 0 0 0 19 10 1 -1 0 3
len X[0]:  15
Class 0:  3551
Class 1:  435


In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
# lấy 10% là test
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print('X_train[0] : ', *X_train[0])

(3587, 15)
(399, 15)
(3587,)
(399,)
X_train[0] :  31 4 1 2 0 1170 0 0 0 20 1 4 -1 0 3


In [36]:
## https://stackoverflow.com/questions/51237635/difference-between-standard-scaler-and-minmaxscaler#:~:text=StandardScaler%20removes%20the%20mean%20and%20scales%20the%20data%20to%20unit%20variance.&text=MinMaxScaler%20rescales%20the%20data%20set,the%20transformed%20number%20of%20households.
## https://stackoverflow.com/questions/49444262/normalize-data-before-or-after-split-of-training-and-testing-data
# Scale sau bước train_test_split để không rò rỉ dữ liệu?

#Scale MinMax
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler
scaler_m = MinMaxScaler()
# scaler_s = StandardScaler()
# scaler_r = RobustScaler()

scaler_m.fit(X_train)
X_train_m = scaler_m.transform(X_train)

# scaler_s.fit(X_train)
# X_train_s = scaler_s.transform(X_train)

# scaler_r.fit(X_train)
# X_train_r = scaler_r.transform(X_train)

In [37]:
print(X_train[:2])
print(X_train_m[:2])
# print(X_train_s[:2])
# print(X_train_r[:2])

[[  31    4    1    2    0 1170    0    0    0   20    1    4   -1    0
     3]
 [  58    1    0    0    0 8218    1    0    0   31    5   10   -1    0
     3]]
[[0.22641509 0.36363636 0.5        0.66666667 0.         0.24051468
  0.         0.         0.         0.63333333 0.09090909 0.27272727
  0.         0.         1.        ]
 [0.73584906 0.09090909 0.         0.         0.         0.82184098
  1.         0.         0.         1.         0.45454545 0.81818182
  0.         0.         1.        ]]


In [68]:
#identity scale minmax
from sklearn.neural_network import MLPClassifier

model_identity = MLPClassifier(max_iter=500, batch_size=200, verbose=True, learning_rate_init=0.01, activation='identity', hidden_layer_sizes=(10, 10), random_state=42).fit(X_train_m, y_train)
# model_identity = MLPClassifier(verbose=True, learning_rate='adaptive', activation='identity', hidden_layer_sizes=(10, 10), max_iter=500, random_state=42).fit(X_train_m, y_train)
y_pred_iden_m = model_identity.predict(X_test)
# 3587/200=17.935

Iteration 1, loss = 0.35394533
Iteration 2, loss = 0.33278773
Iteration 3, loss = 0.32902829
Iteration 4, loss = 0.32625070
Iteration 5, loss = 0.32618354
Iteration 6, loss = 0.32520926
Iteration 7, loss = 0.32290967
Iteration 8, loss = 0.32280295
Iteration 9, loss = 0.32183683
Iteration 10, loss = 0.32285378
Iteration 11, loss = 0.32209219
Iteration 12, loss = 0.32103220
Iteration 13, loss = 0.31906042
Iteration 14, loss = 0.32162778
Iteration 15, loss = 0.31952555
Iteration 16, loss = 0.32163356
Iteration 17, loss = 0.31817926
Iteration 18, loss = 0.31869994
Iteration 19, loss = 0.31790462
Iteration 20, loss = 0.31878025
Iteration 21, loss = 0.31827018
Iteration 22, loss = 0.31806472
Iteration 23, loss = 0.31835469
Iteration 24, loss = 0.32003793
Iteration 25, loss = 0.31845686
Iteration 26, loss = 0.31746849
Iteration 27, loss = 0.31827349
Iteration 28, loss = 0.31736313
Iteration 29, loss = 0.31695642
Iteration 30, loss = 0.31840686
Iteration 31, loss = 0.31767516
Iteration 32, los

In [52]:
#sigmoid scale minmax
model_logistic = MLPClassifier(verbose=True, learning_rate_init=0.01,activation='logistic', hidden_layer_sizes=(10, 10), max_iter=500, random_state=42).fit(X_train_m, y_train)
y_pred_logistic_m = model_logistic.predict(X_test)

Iteration 1, loss = 1.04831270
Iteration 2, loss = 0.59432271
Iteration 3, loss = 0.43096289
Iteration 4, loss = 0.36826812
Iteration 5, loss = 0.35108962
Iteration 6, loss = 0.34795907
Iteration 7, loss = 0.34764583
Iteration 8, loss = 0.34761918
Iteration 9, loss = 0.34745277
Iteration 10, loss = 0.34729384
Iteration 11, loss = 0.34706019
Iteration 12, loss = 0.34666461
Iteration 13, loss = 0.34612350
Iteration 14, loss = 0.34541003
Iteration 15, loss = 0.34443062
Iteration 16, loss = 0.34329711
Iteration 17, loss = 0.34153379
Iteration 18, loss = 0.33965440
Iteration 19, loss = 0.33750807
Iteration 20, loss = 0.33543267
Iteration 21, loss = 0.33289501
Iteration 22, loss = 0.33069653
Iteration 23, loss = 0.32870475
Iteration 24, loss = 0.32695377
Iteration 25, loss = 0.32542615
Iteration 26, loss = 0.32433086
Iteration 27, loss = 0.32364145
Iteration 28, loss = 0.32240715
Iteration 29, loss = 0.32229621
Iteration 30, loss = 0.32159034
Iteration 31, loss = 0.32057395
Iteration 32, los

In [40]:
#tanh scale minmax
model_tanh = MLPClassifier(verbose=True, learning_rate_init=0.01, activation='tanh', hidden_layer_sizes=(10, 10), max_iter=500, random_state=42).fit(X_train_m, y_train)
y_pred_tanh_m = model_tanh.predict(X_test)

Iteration 1, loss = 0.38769062
Iteration 2, loss = 0.34838279
Iteration 3, loss = 0.34096257
Iteration 4, loss = 0.33173115
Iteration 5, loss = 0.32784379
Iteration 6, loss = 0.32359972
Iteration 7, loss = 0.32236160
Iteration 8, loss = 0.31987817
Iteration 9, loss = 0.31972531
Iteration 10, loss = 0.31834698
Iteration 11, loss = 0.31926045
Iteration 12, loss = 0.31856963
Iteration 13, loss = 0.31744847
Iteration 14, loss = 0.31889181
Iteration 15, loss = 0.31700080
Iteration 16, loss = 0.31746218
Iteration 17, loss = 0.31536842
Iteration 18, loss = 0.31538398
Iteration 19, loss = 0.31502485
Iteration 20, loss = 0.31490878
Iteration 21, loss = 0.31461313
Iteration 22, loss = 0.31379618
Iteration 23, loss = 0.31620685
Iteration 24, loss = 0.31523391
Iteration 25, loss = 0.31192168
Iteration 26, loss = 0.31210845
Iteration 27, loss = 0.31247740
Iteration 28, loss = 0.31127182
Iteration 29, loss = 0.31021466
Iteration 30, loss = 0.31111616
Iteration 31, loss = 0.30955996
Iteration 32, los

Iteration 263, loss = 0.24227675
Iteration 264, loss = 0.24129690
Iteration 265, loss = 0.24162651
Iteration 266, loss = 0.24298128
Iteration 267, loss = 0.24283237
Iteration 268, loss = 0.24296018
Iteration 269, loss = 0.24016672
Iteration 270, loss = 0.24032761
Iteration 271, loss = 0.24147797
Iteration 272, loss = 0.23957247
Iteration 273, loss = 0.24192000
Iteration 274, loss = 0.24016473
Iteration 275, loss = 0.23915023
Iteration 276, loss = 0.23806343
Iteration 277, loss = 0.23956304
Iteration 278, loss = 0.23830041
Iteration 279, loss = 0.23848085
Iteration 280, loss = 0.24049485
Iteration 281, loss = 0.24105643
Iteration 282, loss = 0.23970492
Iteration 283, loss = 0.23798975
Iteration 284, loss = 0.23811714
Iteration 285, loss = 0.23795014
Iteration 286, loss = 0.23769384
Iteration 287, loss = 0.23892448
Iteration 288, loss = 0.23989386
Iteration 289, loss = 0.24044600
Iteration 290, loss = 0.23648855
Iteration 291, loss = 0.23633555
Iteration 292, loss = 0.23779956
Iteration 

In [41]:
#relu scale minmax
model_relu = MLPClassifier(verbose=True, learning_rate_init=0.01, activation='relu', hidden_layer_sizes=(10, 10), max_iter=500, random_state=42).fit(X_train_m, y_train)
y_pred_relu_m = model_relu.predict(X_test)

Iteration 1, loss = 0.96050828
Iteration 2, loss = 0.55443566
Iteration 3, loss = 0.35176082
Iteration 4, loss = 0.33480668
Iteration 5, loss = 0.33223207
Iteration 6, loss = 0.32894130
Iteration 7, loss = 0.32703443
Iteration 8, loss = 0.32586133
Iteration 9, loss = 0.32491734
Iteration 10, loss = 0.32440516
Iteration 11, loss = 0.32357996
Iteration 12, loss = 0.32330142
Iteration 13, loss = 0.32280701
Iteration 14, loss = 0.32252181
Iteration 15, loss = 0.32211081
Iteration 16, loss = 0.32263518
Iteration 17, loss = 0.32176169
Iteration 18, loss = 0.32115234
Iteration 19, loss = 0.32091102
Iteration 20, loss = 0.32043199
Iteration 21, loss = 0.31937108
Iteration 22, loss = 0.31764808
Iteration 23, loss = 0.31737160
Iteration 24, loss = 0.31490543
Iteration 25, loss = 0.31330811
Iteration 26, loss = 0.31215796
Iteration 27, loss = 0.31195849
Iteration 28, loss = 0.31105212
Iteration 29, loss = 0.30954111
Iteration 30, loss = 0.31054025
Iteration 31, loss = 0.30998964
Iteration 32, los

In [81]:
# Scale minmax
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print('----------identity')
print('accuracy:\n', accuracy_score(y_test, y_pred_iden_m)*100)
print(confusion_matrix(y_test, y_pred_iden_m, labels=[1, 0]).T)
print(confusion_matrix(y_test, y_pred_iden_m, labels=[0, 1]).T)
# print(pd.crosstab(y_test, y_pred_iden_m))
print(classification_report(y_test, y_pred_iden_m, target_names=['1','0']))

print('----------logistic')
print('accuracy:\n', accuracy_score(y_test, y_pred_logi_m)*100)
print(confusion_matrix(y_test, y_pred_logi_m, labels=[1, 0]))
print(classification_report(y_test, y_pred_logi_m, target_names=['1','0']))

print('----------tanh')
print('accuracy:\n', accuracy_score(y_test, y_pred_tanh_m)*100)
print(confusion_matrix(y_test, y_pred_tanh_m, labels=[1, 0]))
print(classification_report(y_test, y_pred_tanh_m, target_names=['1','0']))

print('----------relu')
print('accuracy:\n', accuracy_score(y_test, y_pred_relu_m)*100)
print(confusion_matrix(y_test, y_pred_relu_m, labels=[1, 0]))
print(classification_report(y_test, y_pred_relu_m, target_names=['1','0']))

#          0   1
# True    33   291
# False   5    70

# 0.1 = 33/(33+291)
# 0.87 = 33/(33+5)

#          1   0
# True    70   5
# False   291  33

# 0.93 = 70/(70+5) 
# 0.19 = 70/(70+291)

----------identity
accuracy:
 25.81453634085213
[[ 33 291]
 [  5  70]]
[[ 70   5]
 [291  33]]
              precision    recall  f1-score   support

           1       0.93      0.19      0.32       361
           0       0.10      0.87      0.18        38

    accuracy                           0.26       399
   macro avg       0.52      0.53      0.25       399
weighted avg       0.85      0.26      0.31       399

----------logistic
accuracy:
 90.47619047619048
[[  0  38]
 [  0 361]]
              precision    recall  f1-score   support

           1       0.90      1.00      0.95       361
           0       0.00      0.00      0.00        38

    accuracy                           0.90       399
   macro avg       0.45      0.50      0.48       399
weighted avg       0.82      0.90      0.86       399

----------tanh
accuracy:
 58.89724310776943
[[ 13  25]
 [139 222]]
              precision    recall  f1-score   support

           1       0.90      0.61      0.73       361
     